In [3]:
from vnstock import Quote
from tenacity import RetryError
from datetime import date, timedelta
import pandas as pd
import os


# === Extract: lấy dữ liệu mới kể từ ngày cuối cùng trong file local ===
# def extract_data(symbol, base_path="data/lake", start_date=None):
#     today = date.today()
#     if start_date == None:
#         start_date = "2020-01-01"

#     # Nếu đã có file, đọc ngày cuối cùng để chỉ lấy incremental
#     file_path = os.path.join(base_path, f"{symbol}.csv")
#     if os.path.exists(file_path):
#           # Get start_date from existed file 
#         try:
#             existing_df = pd.read_csv(file_path)
#             if not existing_df.empty:
#                 last_date = pd.to_datetime(existing_df["time"]).max()
#                 start_date = (last_date + timedelta(days=1)).strftime("%Y-%m-%d")
#         except Exception as e:
#             print(f"Lỗi khi đọc file {symbol}.csv: {e}")

#     print(f"{symbol}: trích dữ liệu từ {start_date} đến {today}")
#       # Call the API 
#     quote = Quote(symbol=symbol, source="VCI")
#     try:
#         df = quote.history(start=start_date, end=today.strftime("%Y-%m-%d"))
#         print(f"Trích xuất {len(df)} dòng mới cho {symbol}")
#     except (ValueError, RetryError):
#         print(f"Không tìm thấy dữ liệu cho {symbol}")
#         df = pd.DataFrame(columns=["time", "open", "high", "low", "close", "volume"])

#     return df

In [6]:
path = os.path.join("data/lake", "AAH.csv")
if os.path.exists(path):
    try:
        existing_df = pd.read_csv(path)
        if not existing_df.empty:
            last_date = pd.to_datetime(existing_df["time"]).max()
            start_date = (last_date + timedelta(days=1)).strftime("%Y-%m-%d")
    except Exception as e:
        print(f"Lỗi khi đọc file AAH.csv: {e}")



In [8]:
print(last_date)
print(start_date)

2025-10-07 00:00:00
2025-10-08


In [4]:


today = date.today()

quote = Quote(symbol='TNV', source="VCI")
try:
    df = quote.history(start='2020-01-01', end=today.strftime("%Y-%m-%d"))
    print(f"Trích xuất {len(df)} dòng mới cho AAH")
except (ValueError, RetryError):
    print(f"Không tìm thấy dữ liệu cho AAH")
    df = pd.DataFrame(columns=["time", "open", "high", "low", "close", "volume"])

Trích xuất 155 dòng mới cho AAH


In [6]:
df

,time,open,high,low,close,volume
0,2025-02-06,12.4,12.4,12.4,12.4,100
1,2025-02-07,14.2,14.2,14.2,14.2,100
2,2025-02-10,16.3,16.3,16.3,16.3,400
3,2025-02-11,18.7,18.7,18.7,18.7,100
4,2025-02-12,21.5,21.5,21.5,21.5,100
...,...,...,...,...,...,...
150,2025-10-03,10.0,10.0,10.0,10.0,1000
151,2025-10-06,10.0,10.0,10.0,10.0,400
152,2025-10-07,10.0,10.3,10.0,10.2,3600
153,2025-10-09,10.3,10.6,10.3,10.6,600


Không tìm thấy dữ liệu cho AAH


0

In [6]:
from src.batch.etl_vnstock import extract_data, load_data_to_minio
from minio import Minio 
from vnstock import Quote

df_new = extract_data("VCI", 'lake', start_date="2025-10-10")

minio_client = Minio(
    "localhost:9000",
    access_key="vnstock",
    secret_key="12345678",
    secure=False
)

load_data_to_minio(df_new, "VCI", minio_client, "vnstock", mode="append")


VCI: trích dữ liệu từ 2025-10-10 đến 2025-10-29
Trích xuất 14 dòng mới cho VCI
Append 14 dòng mới vào VCI.parquet
Upload thành công VCI.parquet lên MinIO ✅
